In [1]:
import cPickle as pickle
from multiprocessing import Pool

import numpy as np

import dynamic_benchmark

In [2]:
""" Calculate the Pareto set solution
    
    INPUT:
        offset: 0 implies this is the global Pareto set.
        x1: float value to indicate the value of the first parameter
        t: time variable
        
    OUTPUT:
        output: Pareto optimal point (x1, ...)
"""
def pareto_set_2obj(offset, x1, t):
    output = [x1]
    output.extend([offset + np.sin(0.5*np.pi*(t-x1)) for e in range(20)])
    return output

def pareto_set_3obj(offset, x1, x2, t):
    output = [x1, x2]
    output.extend([offset + np.sin(0.5*np.pi*(t-x1)) for e in range(20)])
    return output

In [3]:
""" Efficient Pareto Filtering method. The code is obtained from: 
    http://code.activestate.com/recipes/578287-multidimensional-pareto-front
"""
def dominates(row, candidate_row):
    return sum([row[x] <= candidate_row[x] for x in range(len(row))]) == len(row)    


def pareto_filter(input_points, dominates=dominates):
    candidate_rowNr  = 0
    pareto_points    = set()
    dominated_points = set()
    while True:
        candidate_row = input_points[candidate_rowNr]
        input_points.remove(candidate_row)
        rowNr = 0
        non_dominated = True
        while len(input_points) != 0 and rowNr < len(input_points):
            row = input_points[rowNr]
            if dominates(candidate_row, row):
                # If it is worse on all features remove the row from the array
                input_points.remove(row)
                dominated_points.add(tuple(row))
            elif dominates(row, candidate_row):
                non_dominated = False
                dominated_points.add(tuple(candidate_row))
                rowNr += 1
            else:
                rowNr += 1

        if non_dominated:
            # add the non-dominated point to the Pareto frontier
            pareto_points.add(tuple(candidate_row))

        if len(input_points) == 0:
            break
    return pareto_points, dominated_points

In [4]:
# create Pareto set for 2-objective problem
ps_dict_2obj = dict()

for t in range(41):
    time_value = 0.1*t
    x1 = np.linspace(0, 1, 1000)
    x_array = list([pareto_set_2obj(0, _x1, time_value) for _x1 in x1])
    ps_dict_2obj[str(time_value)] = x_array

In [5]:
# create Pareto set for 3-objective problem
ps_dict_3obj = dict()

for t in range(41):
    time_value = 0.1*t
    x1 = np.linspace(0, 1, 100)
    x2 = np.linspace(0, 1, 100)
    x1, x2 = np.meshgrid(x1, x2)
    x1 = x1.reshape(-1)
    x2 = x2.reshape(-1)
    x_array = list([pareto_set_3obj(0, _x1, _x2, time_value) for _x1, _x2 in zip(x1, x2)])
    ps_dict_3obj[str(time_value)] = x_array

Function to create Pareto front/set for each problem (GTA1-GTA8):

In [6]:
# create Pareto front for two-objective problem
def pareto_points(args):
    ps_dict, problem = args
    func = getattr(dynamic_benchmark, problem)
    pf_dict = dict()

    for time_value in ps_dict:
        obj_array = list([func(x, float(time_value)) for x in ps_dict[time_value]])
        pf, _ = pareto_filter(obj_array)
        pf = set(pf)
        ps = [x for x in ps_dict[time_value] if tuple(func(x, float(time_value))) in pf]
        pf_dict[time_value] = (pf, ps)
    
    with open("pareto/{}.p".format(problem), "wb") as fhandle:
        pickle.dump(pf_dict, fhandle)

Dictionary to store the optimal solutions:

In [7]:
input_list = [(ps_dict_2obj, "DB{}a".format(i+1)) for i in range(8)]
input_list += [(ps_dict_3obj, "DB{}a".format(i+9)) for i in range(4)]

p = Pool(8)
p.map(pareto_points, input_list);